In [8]:
""" Ingest data on Google Earth Engine
-------------------------------------------------------------------------------
This notebook will upload the geotiff files from the Google Cloud Storage to
the WRI/aqueduct earthengine bucket. 

Requirements:
    Authorize earthengine by running in your terminal: earthengine 
                                                       authenticate

    you need to have access to the WRI-Aquaduct (yep a Google employee made a
    typo) bucket to ingest the data. Rutger can grant access to write to this 
    folder. 

    Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

Code follows the Google for Python Styleguide. Exception are the scripts that 
use earth engine since this is camelCase instead of underscore.


Author: Rutger Hofste
Date: 20170802
Kernel: python36
Docker: rutgerhofste/gisdocker:ubuntu16.04

Args:

    SCRIPT_NAME (string) : Script name


Returns:


"""

# Input Parameters

SCRIPT_NAME = "Y2017M08D02_RH_Ingest_GCS_EE_V02"
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V02/"
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V08"

# Output Parameters


In [9]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2018M04D04', 'UTC 17:02')


'2.7.14 |Anaconda, Inc.| (default, Dec  7 2017, 17:05:42) \n[GCC 7.2.0]'

In [10]:
# Imports
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

In [12]:
# ETL

command = ("earthengine create folder %s") %EE_BASE
print(command)
subprocess.check_output(command,shell=True)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V08


''

In [31]:
# Functions

def split_key(key):
    """ Split key into dictionary
    -------------------------------------------------------------------------------
    WARNING: This function is dependant on the name convention of PCRGLOBWB
    Do not use with other keys
    
    Args:
        key (string) : key containing information about parameter, year month etc.
        
    Returns:
        out_dict (dictionary): Dictionary containing all information contained
                               in key.      

    """
    
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    file_name = prefix.split("/")[-1]
    parameter = file_name[:-12]
    month = file_name[-2:] #can also do this with regular expressions if you like
    year = file_name[-7:-3]
    identifier = file_name[-11:-8]
    out_dict = {"file_name":file_name,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return out_dict

def split_parameter(parameter):
    """Split parameter 
    -------------------------------------------------------------------------------
    WARNING: This function is dependant on the name convention of PCRGLOBWB
    Do not use with other keys
    
    Args:
        parameter (string) : parameter string.
    
    Returns:
        out_dict (dictionary) : dictionary containing all information contained
                                in parameter key.
    
    """
    
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    out_dict = dict(zip(keys, values))
    out_dict["parameter"] = parameter
    return out_dict

def upload_geotiff_to_EE_imageCollection(geotiff_gcs_path,output_ee_asset_id,properties):
    """Upload geotiff to earthengine image collection
    -------------------------------------------------------------------------------
    
    Ingest a geotiff to earth engine imageCollection and set metadata. A dictionary
    of properties will be used to define the metadata of the image.
    
    Args:
        geotiff_gcs_path (string) : Google Cloud Storage path of geotiff.
        output_ee_asset_id (string) : Earth Engine output asset id. Full path 
                                      including imageCollection asset id.
        properties (dictionary) : Dictionary with metadata. the 'nodata_value' key
                                  can be used to set a NoData Value.
        
    
    Returns:
        command (string) : command string parsed to subprocess module.
        
    
    TODO:
    update function to work with dictionary of properties
    
    """
    
    
    
    
    
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p month=%s -p parameter=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.month,row.parameter,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        response = subprocess.check_output(command, shell=True)
        outDict = {"command":command,"response":response,"error":0}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        pass
    except:
        try:
            outDict = {"command":command,"response":response,"error":1}
        except:
            outDict = {"command":command,"response":-9999,"error":2}
        df_errors2 = pd.DataFrame(outDict,index=[index])
        print("error")
    return df_errors2


def dictionary_to_upload_command(d):
    """ Convert a dictionary to command that can be appended to upload command
    -------------------------------------------------------------------------------
    
    Args:
        d (dictionary) : Dictionary with metadata.
    
    Returns:
        command (string) : string to append to upload string.    
    
    """
    command = ""
    for key, value in d.items():
            
        if key == "nodata_value":
            command = command + " --nodata_value={}".format(value)
        else:
            command = command + " -p {}={}".format(key,value)

    return command

def create_imageCollection(ic_id):
    """ Creates an imageCollection using command line
    -------------------------------------------------------------------------------
    Args:
        ic_id (string) : asset_id of image Collection.
        
    Returns:
        command (string) : command parsed to subprocess module 
        result (string) : subprocess result 
        
    """
    command = "earthengine create collection {}".format(ic_id)
    result = subprocess.check_output(command,shell=True)
    return command, result


## Script

In [15]:
command = "/opt/google-cloud-sdk/bin/gsutil ls {}".format(GCS_BASE)
keys = subprocess.check_output(command,shell=True)
keys = keys.decode('UTF-8').splitlines()

In [24]:
df = pd.DataFrame()
i = 0
for key in keys:
    i = i+1
    out_dict = split_key(key)
    df2 = pd.DataFrame(out_dict,index=[i])
    df = df.append(df2)    

In [25]:
df.head()

,extension,file_name,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [26]:
df.tail()

,extension,file_name,identifier,month,parameter,year
9286,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
9287,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
9288,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
9289,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
9290,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [27]:
df.shape

(9290, 6)

In [28]:
parameters = df.parameter.unique()

In [29]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

In [33]:
for parameter in parameters:
    ic_id = EE_BASE + "/" + parameter
    command, result = create_imageCollection(ic_id)
    print(command)
    

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [36]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    out_dict_parameter = split_parameter(parameter)
    df_parameter2 = pd.DataFrame(out_dict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [37]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [38]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,runoff,global_historical_runoff_month_mmonth_5min_195...,5min,historical,2014,1958,month,mmonth
24,global,runoff,global_historical_runoff_year_myear_5min_1958_...,5min,historical,2014,1958,year,myear
25,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [39]:
df_parameter.shape

(25, 9)

In [40]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [41]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [42]:
df_complete.head()

,extension,file_name,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [43]:
df_complete.tail()

,extension,file_name,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
9285,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
9286,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
9287,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
9288,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
9289,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [27]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [46]:
df_errors = pd.DataFrame()
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/9289.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    
    upload_geotiff_to_EE_imageCollection(geotiff_gcs_path,output_ee_asset_id,properties)
    
    #df_errors2 = uploadEE(index,row)
    #df_errors = df_errors.append(df_errors2)
    
    

(0, '0.00', 'elapsed: ', '0:00:00.004201')
(1, '0.01', 'elapsed: ', '0:00:00.004417')
(2, '0.02', 'elapsed: ', '0:00:00.004783')
(3, '0.03', 'elapsed: ', '0:00:00.005032')
(4, '0.04', 'elapsed: ', '0:00:00.005278')
(5, '0.05', 'elapsed: ', '0:00:00.005467')
(6, '0.06', 'elapsed: ', '0:00:00.005699')
(7, '0.08', 'elapsed: ', '0:00:00.005908')
(8, '0.09', 'elapsed: ', '0:00:00.006117')
(9, '0.10', 'elapsed: ', '0:00:00.006336')
(10, '0.11', 'elapsed: ', '0:00:00.006534')
(11, '0.12', 'elapsed: ', '0:00:00.006745')
(12, '0.13', 'elapsed: ', '0:00:00.006972')
(13, '0.14', 'elapsed: ', '0:00:00.007165')
(14, '0.15', 'elapsed: ', '0:00:00.007370')
(15, '0.16', 'elapsed: ', '0:00:00.007578')
(16, '0.17', 'elapsed: ', '0:00:00.007779')
(17, '0.18', 'elapsed: ', '0:00:00.007986')
(18, '0.19', 'elapsed: ', '0:00:00.008190')
(19, '0.20', 'elapsed: ', '0:00:00.008397')
(20, '0.22', 'elapsed: ', '0:00:00.008599')
(21, '0.23', 'elapsed: ', '0:00:00.008806')
(22, '0.24', 'elapsed: ', '0:00:00.009008'

(852, '9.17', 'elapsed: ', '0:00:00.204587')
(853, '9.18', 'elapsed: ', '0:00:00.206705')
(854, '9.19', 'elapsed: ', '0:00:00.206921')
(855, '9.20', 'elapsed: ', '0:00:00.207123')
(856, '9.22', 'elapsed: ', '0:00:00.207400')
(857, '9.23', 'elapsed: ', '0:00:00.207583')
(858, '9.24', 'elapsed: ', '0:00:00.207751')
(859, '9.25', 'elapsed: ', '0:00:00.207968')
(860, '9.26', 'elapsed: ', '0:00:00.208222')
(861, '9.27', 'elapsed: ', '0:00:00.208475')
(862, '9.28', 'elapsed: ', '0:00:00.208707')
(863, '9.29', 'elapsed: ', '0:00:00.208969')
(864, '9.30', 'elapsed: ', '0:00:00.209179')
(865, '9.31', 'elapsed: ', '0:00:00.209415')
(866, '9.32', 'elapsed: ', '0:00:00.209650')
(867, '9.33', 'elapsed: ', '0:00:00.209984')
(868, '9.34', 'elapsed: ', '0:00:00.210231')
(869, '9.36', 'elapsed: ', '0:00:00.210466')
(870, '9.37', 'elapsed: ', '0:00:00.210720')
(871, '9.38', 'elapsed: ', '0:00:00.210963')
(872, '9.39', 'elapsed: ', '0:00:00.211207')
(873, '9.40', 'elapsed: ', '0:00:00.211408')
(874, '9.4

(1623, '17.47', 'elapsed: ', '0:00:00.407268')
(1624, '17.48', 'elapsed: ', '0:00:00.409009')
(1625, '17.49', 'elapsed: ', '0:00:00.409165')
(1626, '17.50', 'elapsed: ', '0:00:00.409394')
(1627, '17.52', 'elapsed: ', '0:00:00.409604')
(1628, '17.53', 'elapsed: ', '0:00:00.409894')
(1629, '17.54', 'elapsed: ', '0:00:00.410071')
(1630, '17.55', 'elapsed: ', '0:00:00.410278')
(1631, '17.56', 'elapsed: ', '0:00:00.410513')
(1632, '17.57', 'elapsed: ', '0:00:00.410744')
(1633, '17.58', 'elapsed: ', '0:00:00.410933')
(1634, '17.59', 'elapsed: ', '0:00:00.411190')
(1635, '17.60', 'elapsed: ', '0:00:00.411421')
(1636, '17.61', 'elapsed: ', '0:00:00.411657')
(1637, '17.62', 'elapsed: ', '0:00:00.411938')
(1638, '17.63', 'elapsed: ', '0:00:00.412116')
(1639, '17.64', 'elapsed: ', '0:00:00.412357')
(1640, '17.66', 'elapsed: ', '0:00:00.412600')
(1641, '17.67', 'elapsed: ', '0:00:00.412812')
(1642, '17.68', 'elapsed: ', '0:00:00.413013')
(1643, '17.69', 'elapsed: ', '0:00:00.413316')
(1644, '17.70

(2396, '25.79', 'elapsed: ', '0:00:00.609732')
(2397, '25.80', 'elapsed: ', '0:00:00.610032')
(2398, '25.82', 'elapsed: ', '0:00:00.611593')
(2399, '25.83', 'elapsed: ', '0:00:00.611833')
(2400, '25.84', 'elapsed: ', '0:00:00.612104')
(2401, '25.85', 'elapsed: ', '0:00:00.612274')
(2402, '25.86', 'elapsed: ', '0:00:00.612447')
(2403, '25.87', 'elapsed: ', '0:00:00.612738')
(2404, '25.88', 'elapsed: ', '0:00:00.613038')
(2405, '25.89', 'elapsed: ', '0:00:00.613288')
(2406, '25.90', 'elapsed: ', '0:00:00.613508')
(2407, '25.91', 'elapsed: ', '0:00:00.613749')
(2408, '25.92', 'elapsed: ', '0:00:00.614065')
(2409, '25.93', 'elapsed: ', '0:00:00.614367')
(2410, '25.94', 'elapsed: ', '0:00:00.614603')
(2411, '25.96', 'elapsed: ', '0:00:00.614843')
(2412, '25.97', 'elapsed: ', '0:00:00.615075')
(2413, '25.98', 'elapsed: ', '0:00:00.615372')
(2414, '25.99', 'elapsed: ', '0:00:00.615583')
(2415, '26.00', 'elapsed: ', '0:00:00.615757')
(2416, '26.01', 'elapsed: ', '0:00:00.615929')
(2417, '26.02

(3077, '33.13', 'elapsed: ', '0:00:00.811929')
(3078, '33.14', 'elapsed: ', '0:00:00.813622')
(3079, '33.15', 'elapsed: ', '0:00:00.813911')
(3080, '33.16', 'elapsed: ', '0:00:00.814158')
(3081, '33.17', 'elapsed: ', '0:00:00.814338')
(3082, '33.18', 'elapsed: ', '0:00:00.814499')
(3083, '33.19', 'elapsed: ', '0:00:00.814730')
(3084, '33.20', 'elapsed: ', '0:00:00.814970')
(3085, '33.21', 'elapsed: ', '0:00:00.815244')
(3086, '33.22', 'elapsed: ', '0:00:00.815543')
(3087, '33.23', 'elapsed: ', '0:00:00.815787')
(3088, '33.24', 'elapsed: ', '0:00:00.816093')
(3089, '33.25', 'elapsed: ', '0:00:00.816331')
(3090, '33.27', 'elapsed: ', '0:00:00.816574')
(3091, '33.28', 'elapsed: ', '0:00:00.816809')
(3092, '33.29', 'elapsed: ', '0:00:00.817053')
(3093, '33.30', 'elapsed: ', '0:00:00.817349')
(3094, '33.31', 'elapsed: ', '0:00:00.817602')
(3095, '33.32', 'elapsed: ', '0:00:00.817828')
(3096, '33.33', 'elapsed: ', '0:00:00.818112')
(3097, '33.34', 'elapsed: ', '0:00:00.818351')
(3098, '33.35

...

(5981, '64.39', 'elapsed: ', '0:00:01.607222')
(5982, '64.40', 'elapsed: ', '0:00:01.626297')
(5983, '64.41', 'elapsed: ', '0:00:01.626510')
(5984, '64.42', 'elapsed: ', '0:00:01.626758')
(5985, '64.43', 'elapsed: ', '0:00:01.627001')
(5986, '64.44', 'elapsed: ', '0:00:01.627186')
(5987, '64.45', 'elapsed: ', '0:00:01.627428')
(5988, '64.46', 'elapsed: ', '0:00:01.627737')
(5989, '64.47', 'elapsed: ', '0:00:01.627977')
(5990, '64.48', 'elapsed: ', '0:00:01.628258')
(5991, '64.50', 'elapsed: ', '0:00:01.628466')
(5992, '64.51', 'elapsed: ', '0:00:01.628767')
(5993, '64.52', 'elapsed: ', '0:00:01.628965')
(5994, '64.53', 'elapsed: ', '0:00:01.629257')
(5995, '64.54', 'elapsed: ', '0:00:01.629492')
(5996, '64.55', 'elapsed: ', '0:00:01.629733')
(5997, '64.56', 'elapsed: ', '0:00:01.629978')
(5998, '64.57', 'elapsed: ', '0:00:01.630218')
(5999, '64.58', 'elapsed: ', '0:00:01.630462')
(6000, '64.59', 'elapsed: ', '0:00:01.630687')
(6001, '64.60', 'elapsed: ', '0:00:01.630865')
(6002, '64.61

(6738, '72.54', 'elapsed: ', '0:00:01.826230')
(6739, '72.55', 'elapsed: ', '0:00:01.828082')
(6740, '72.56', 'elapsed: ', '0:00:01.828305')
(6741, '72.57', 'elapsed: ', '0:00:01.828508')
(6742, '72.58', 'elapsed: ', '0:00:01.828792')
(6743, '72.59', 'elapsed: ', '0:00:01.828970')
(6744, '72.60', 'elapsed: ', '0:00:01.829143')
(6745, '72.61', 'elapsed: ', '0:00:01.829370')
(6746, '72.62', 'elapsed: ', '0:00:01.829630')
(6747, '72.63', 'elapsed: ', '0:00:01.829875')
(6748, '72.65', 'elapsed: ', '0:00:01.830114')
(6749, '72.66', 'elapsed: ', '0:00:01.830413')
(6750, '72.67', 'elapsed: ', '0:00:01.830664')
(6751, '72.68', 'elapsed: ', '0:00:01.830920')
(6752, '72.69', 'elapsed: ', '0:00:01.831154')
(6753, '72.70', 'elapsed: ', '0:00:01.831402')
(6754, '72.71', 'elapsed: ', '0:00:01.831611')
(6755, '72.72', 'elapsed: ', '0:00:01.831854')
(6756, '72.73', 'elapsed: ', '0:00:01.832098')
(6757, '72.74', 'elapsed: ', '0:00:01.832290')
(6758, '72.75', 'elapsed: ', '0:00:01.832588')
(6759, '72.76

(7480, '80.53', 'elapsed: ', '0:00:02.024039')
(7481, '80.54', 'elapsed: ', '0:00:02.042402')
(7482, '80.55', 'elapsed: ', '0:00:02.042588')
(7483, '80.56', 'elapsed: ', '0:00:02.042816')
(7484, '80.57', 'elapsed: ', '0:00:02.043000')
(7485, '80.58', 'elapsed: ', '0:00:02.043230')
(7486, '80.59', 'elapsed: ', '0:00:02.043465')
(7487, '80.60', 'elapsed: ', '0:00:02.043697')
(7488, '80.61', 'elapsed: ', '0:00:02.043941')
(7489, '80.62', 'elapsed: ', '0:00:02.044180')
(7490, '80.63', 'elapsed: ', '0:00:02.044419')
(7491, '80.64', 'elapsed: ', '0:00:02.044646')
(7492, '80.65', 'elapsed: ', '0:00:02.044876')
(7493, '80.67', 'elapsed: ', '0:00:02.045115')
(7494, '80.68', 'elapsed: ', '0:00:02.045346')
(7495, '80.69', 'elapsed: ', '0:00:02.045647')
(7496, '80.70', 'elapsed: ', '0:00:02.045999')
(7497, '80.71', 'elapsed: ', '0:00:02.046236')
(7498, '80.72', 'elapsed: ', '0:00:02.046477')
(7499, '80.73', 'elapsed: ', '0:00:02.046705')
(7500, '80.74', 'elapsed: ', '0:00:02.046963')
(7501, '80.75

(8239, '88.70', 'elapsed: ', '0:00:02.242234')
(8240, '88.71', 'elapsed: ', '0:00:02.244065')
(8241, '88.72', 'elapsed: ', '0:00:02.244278')
(8242, '88.73', 'elapsed: ', '0:00:02.244481')
(8243, '88.74', 'elapsed: ', '0:00:02.244766')
(8244, '88.75', 'elapsed: ', '0:00:02.244939')
(8245, '88.76', 'elapsed: ', '0:00:02.245109')
(8246, '88.77', 'elapsed: ', '0:00:02.245338')
(8247, '88.78', 'elapsed: ', '0:00:02.245599')
(8248, '88.79', 'elapsed: ', '0:00:02.245839')
(8249, '88.80', 'elapsed: ', '0:00:02.246058')
(8250, '88.81', 'elapsed: ', '0:00:02.246267')
(8251, '88.83', 'elapsed: ', '0:00:02.246497')
(8252, '88.84', 'elapsed: ', '0:00:02.246737')
(8253, '88.85', 'elapsed: ', '0:00:02.246969')
(8254, '88.86', 'elapsed: ', '0:00:02.247207')
(8255, '88.87', 'elapsed: ', '0:00:02.247442')
(8256, '88.88', 'elapsed: ', '0:00:02.247670')
(8257, '88.89', 'elapsed: ', '0:00:02.247960')
(8258, '88.90', 'elapsed: ', '0:00:02.248203')
(8259, '88.91', 'elapsed: ', '0:00:02.248372')
(8260, '88.92

(8979, '96.66', 'elapsed: ', '0:00:02.437800')
(8980, '96.67', 'elapsed: ', '0:00:02.455290')
(8981, '96.68', 'elapsed: ', '0:00:02.455540')
(8982, '96.70', 'elapsed: ', '0:00:02.455782')
(8983, '96.71', 'elapsed: ', '0:00:02.456011')
(8984, '96.72', 'elapsed: ', '0:00:02.456241')
(8985, '96.73', 'elapsed: ', '0:00:02.456531')
(8986, '96.74', 'elapsed: ', '0:00:02.456845')
(8987, '96.75', 'elapsed: ', '0:00:02.457076')
(8988, '96.76', 'elapsed: ', '0:00:02.457301')
(8989, '96.77', 'elapsed: ', '0:00:02.457530')
(8990, '96.78', 'elapsed: ', '0:00:02.457746')
(8991, '96.79', 'elapsed: ', '0:00:02.457944')
(8992, '96.80', 'elapsed: ', '0:00:02.458155')
(8993, '96.81', 'elapsed: ', '0:00:02.458384')
(8994, '96.82', 'elapsed: ', '0:00:02.458627')
(8995, '96.83', 'elapsed: ', '0:00:02.458909')
(8996, '96.85', 'elapsed: ', '0:00:02.459074')
(8997, '96.86', 'elapsed: ', '0:00:02.459292')
(8998, '96.87', 'elapsed: ', '0:00:02.459511')
(8999, '96.88', 'elapsed: ', '0:00:02.459732')
(9000, '96.89

In [37]:
!mkdir /volumes/data/temp

mkdir: cannot create directory '/volumes/data/temp': File exists


In [38]:
df_errors.to_csv("/volumes/data/temp/df_errors.csv")

In [39]:
!aws s3 cp  /volumes/data/temp/df_errors.csv s3://wri-projects/Aqueduct30/temp/df_errors.csv

upload: ../../../../data/temp/df_errors.csv to s3://wri-projects/Aqueduct30/temp/df_errors.csv


Retry the ones with errors

In [40]:
df_retry = df_errors.loc[df_errors['error'] != 0]

In [41]:
for index, row in df_retry.iterrows():
    response = subprocess.check_output(row.command, shell=True)
    

In [53]:
uniques = df_errors["error"].unique()

In [ ]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)